# 5. MEMORY
랭체인에는 5가지 정도 종류의 메모리가 있는데 각자 저장 방식도 다르고 각자만의 장단점이 있다.
`챗봇에 메모리를 추가하지 않으면 챗봇은 아무것도 기억할 수 없다``.
유저가 자신의 이름을 말하거나 이전 질무에 이어지는 질문을 해도, 챗봇은 메모리, 즉 기억력이 없기 때문에
대화를 이해할 수 있는 능력이 없다.
오픈API에서 제공하는 기본 API는 랭체인 없이 사용할 수 있는데, 메모리를 지원하지 않는다.
한마디로 stateless.
즉, `모델에게 어떤 말을 건네면 모델은 답을 한 직후에 모든 대화 내용을 잊어버리게 된다.`
아무런 내용도 저장하지 않는다는 말이다.
챗GPT에는 메모리가 탑재되어 있기 때문에 실제로 어떤 사람과 얘기하고 있다는 느낌을 들게한다.
챗봇이 이전의 대화 내용이나 질문을 기억하고 답할 수 있으므로...

### 강의 내용
일단 각 메모리의 종류를 알아보고 차이점을 살펴본 후,
랭체인에 메모리를 탑재시키는 방법을 배워보자.

# 5.0 ConversationBufferMemory
첫번째 메모리는 `Conversation Buffer Memory`라고하는데, 이메모리는 되게 단순하다.
그냥 단순히 이전 대화 내용 전체를 저장하는 메모리이다.
### 단점
대화 내용이 길어질수록 `메모리도 계속 커지므로 비효율적`이다.
위에서 설명했듯이 `모델 자체에는 메모리`가 없다. 
그래서 우리가 모델에게 요청을 보낼 때 `이전 대화 기록 전체를 같이 보내야 된다`.
그래야지만 모델이 전에 일어났던 대화를 보고 이해를 할 수 있다.
유저와 AI의 대화가 길어질수록 우리가 모델에게 `매번 보내야 될 대화 기록이 길어진다`는 뜻이다.
상당히 `비효율적`이고 당연히 `비용도 많이 발생`한다.

비효율적이긴 하지만, 그래도 제일 이해하기 쉬운 메모리니까 한번 살펴보자.


In [7]:
from langchain.memory import ConversationBufferMemory

# memory = ConversationBufferMemory()
memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input": "Hi!"}, {"output": "How are you?"})

memory.load_memory_variables({})




{'history': [HumanMessage(content='Hi!'), AIMessage(content='How are you?')]}

In [8]:
memory.save_context({"input": "Hi!"}, {"output": "How are you?"})

memory.load_memory_variables({})
# 아래와 같에 메모리에 모든 이력이 쌓이므로 대화가 길어질수록 비효율적이다.

{'history': [HumanMessage(content='Hi!'),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi!'),
  AIMessage(content='How are you?')]}

# 5.1 ConversationBufferWindowMemory
5.0 에서는 ConversationBufferMemory에 대해서 배웠다.
ConversationBufferMemory와 다르게 ConversationBufferWindowMemory는 `대화의 특정 부분만을 저장하는 메모리`이다.
예를 들어 가장 `최근 5개의 메시지만 저장`한다고 하면 6번째 메시지가 추가되었을 때,
`가장 오래된 메시지는 버려지는 방식`이다.

ConversationBufferWindowMemory는 `대화의 특정 부분, 저장범위는 우리가 직접 설정` 할 수 있다.
### 장점
메모리를 특정 크기로 유지할 수 있다는게 이메모리의 큰 장점이다.
이 메모리의 장점은 모든 대화 내용을 저장하지 않아도 된다는 점
### 단점
챗봇이 `전체 대화`가 아닌 `최근 대화에만 집중`한다는 것이다.


In [9]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

add_message(1, 1)


In [10]:
add_message(2, 2)
add_message(3, 3)
add_message(4, 4)


In [11]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [12]:
add_message(5, 5)
memory.load_memory_variables({})


{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

위 결과를 보면 마지막에 5번쨰 메모리를 추가하면 `첫번째 메모리 데이터가 제거`된 것을 알수 있다.
`메모리의 사이즈가 계속 늘어나지 않는다는게 장점`이지만,
`단점은 챗봇이 예전 대화를 기억하기 힘들다는 점`이다.
`최근에 일어난 대화에만 집중`한다는 것이다.
